In [1]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

# Enable autoreloading if import packages are changed
%load_ext autoreload
%autoreload 2

<IPython.core.display.Javascript object>

In [2]:
import os
import json

PROJECT_DIR = os.getcwd()

In [3]:
import json
from src.utils import utils
import trimesh
import numpy as np
from scipy.spatial import KDTree

from src.eval import chamfer_distance, crop_points_to_bbox, \
    compute_transformation_error, pose_estimate

### Evaluation using eval.py (stage 1)
The commands below are the same as the provided eval.py's `main()`

In [12]:
dataset_name = "box_v4_stage1"
dataset_info = utils.get_dataset_info(dataset_name, PROJECT_DIR)
dataset_info["data"]

f_gt = open(os.path.join(dataset_info["data"]["path"], "gt_camera_parameters.json"), "r")
camera_param_gt = json.load(f_gt)

In [13]:
eps = 1e-8

f_gt = open(os.path.join(dataset_info["data"]["path"], "gt_camera_parameters.json"), "r")
f_predicted = open(os.path.join(dataset_info["data"]["path"], "estimated_camera_parameters.json"), "r")

camera_param_gt = json.load(f_gt)
camera_param_predicted = json.load(f_predicted)

scale = 0
cameras = camera_param_gt["extrinsics"].keys()
for camera in cameras:
    if camera == "00000.jpg":
        continue

    predicted_camera1 = np.array(camera_param_predicted["extrinsics"][camera])
    predicted_camera2 = np.array(camera_param_gt["extrinsics"][camera])
    scale += (np.linalg.norm(predicted_camera1[:3, 3]) / (np.linalg.norm(predicted_camera2[:3, 3] + eps)))

scale /= (len(cameras) - 1)

pcd_gt = trimesh.load(os.path.join(dataset_info["data"]["path"], "gt_points.ply"))
gt_points = np.array(pcd_gt.vertices, dtype = np.float32)

pcd_test = trimesh.load(os.path.join(dataset_info["data"]["path"], "estimated_points.ply")) # Give path to your .ply here (as above)
test_points = np.array(pcd_test.vertices, dtype = np.float32) / scale

bb = [
    np.array([-4.523355, -1.264923,  0.198537]), 
    np.array([4.976645, 2.235077, 9.698537])
] # Only for box evaluation, do not change

gt_points = crop_points_to_bbox(gt_points, bb)
test_points = crop_points_to_bbox(test_points, bb)
rotation_error, translation_error = pose_estimate(camera_param_gt["extrinsics"], camera_param_predicted["extrinsics"], scale)

print("Rotation error:", round(rotation_error, 2))
print("Translation error:", round(translation_error, 3))
print("Chamfer distance between pointclouds:", chamfer_distance(gt_points, test_points))

Rotation error: 2.12
Translation error: 6.574
Chamfer distance between pointclouds: 0.7185401029350127


### Evaluation using eval.py (stage 2)
The commands below are the same as the provided eval.py's `main()`

In [8]:
dataset_name = "milk_stage2"
dataset_info = utils.get_dataset_info(dataset_name, PROJECT_DIR)
dataset_info["data"]

{'path': '/home/yudzyoga/Documents/Courses/2ndSemester/3DCV/Assignments/github/3dcv_group_10/Project/Submission/dataset/milk_stage2',
 'image': '/home/yudzyoga/Documents/Courses/2ndSemester/3DCV/Assignments/github/3dcv_group_10/Project/Submission/dataset/milk_stage2/images',
 'correspondence': '/home/yudzyoga/Documents/Courses/2ndSemester/3DCV/Assignments/github/3dcv_group_10/Project/Submission/dataset/milk_stage2/correspondences'}

In [9]:
def mesh_error(mesh, mesh_points, test_points, num_points):
    tree = KDTree(test_points)
    dist_mesh_points = tree.query(mesh_points)[0]

    dist_points_mesh = trimesh.proximity.closest_point(mesh, test_points)[1]

    return 0.5 * (np.mean(dist_points_mesh) + np.mean(dist_mesh_points))

eps = 1e-8

f_gt = open(os.path.join(dataset_info["data"]["path"], "gt_camera_parameters.json"), "r")
f_predicted = open(os.path.join(dataset_info["data"]["path"], "estimated_camera_parameters.json"), "r")

# f_gt = open("milk/gt_camera_parameters.json", "r")
# f_predicted = open("milk/estimated_camera_parameters.json", "r")

num_points_sample = 1000

camera_param_gt = json.load(f_gt)
camera_param_predicted = json.load(f_predicted)

scale = 0
cameras = camera_param_gt["extrinsics"].keys()
for camera in cameras:
    if camera == "00000.jpg":
        continue

    predicted_camera1 = np.array(camera_param_predicted["extrinsics"][camera])
    predicted_camera2 = np.array(camera_param_gt["extrinsics"][camera])

    scale += (np.linalg.norm(predicted_camera1[:3, 3]) / (np.linalg.norm(predicted_camera2[:3, 3] + eps)))
scale /= (len(cameras) - 1)


# mesh_gt = trimesh.load("milk/gt_mesh.ply")
mesh_gt = trimesh.load(os.path.join(dataset_info["data"]["path"], "gt_mesh.ply"))
gt_points = trimesh.sample.sample_surface_even(mesh_gt, num_points_sample, seed = 42)[0]

# pcd_test = trimesh.load("milk/estimated_points.ply") # Give path to your .ply here (as above)
pcd_test = trimesh.load(os.path.join(dataset_info["data"]["path"], "estimated_points.ply"))
test_points = np.array(pcd_test.vertices, dtype = np.float32) / scale

bb = [
        np.array([-0.5, -0.15,  0.1]), 
        np.array([0.5, 0.4, 1.1])
    ] # Only for milk evaluation, do not change

test_points = crop_points_to_bbox(test_points, bb)

rotation_error, translation_error = pose_estimate(camera_param_gt["extrinsics"], camera_param_predicted["extrinsics"], scale)

print("Rotation error:", round(rotation_error, 2))
print("Translation error:", round(translation_error, 3))
# print("Mesh error:", mesh_error(mesh_gt, gt_points, test_points, num_points_sample))

Rotation error: 1.37
Translation error: 0.444
